In [20]:
import pandas as pd
import numpy as np
import os, sys
from IPython.display import display
from IPython.display import Image as _Imgdis
from numpy import asarray
from matplotlib import pyplot

df = pd.read_csv("Bittrex_BTCUSD_1h.csv", header=1)
df2 = pd.read_csv("Bittrex_ETHUSD_1h.csv",header=1)
df3 = pd.read_csv("Bittrex_LTCUSD_1h.csv",header=1)

df = pd.concat([df, df2, df3])
df.reset_index(drop=True, inplace=True)

df["open_change"] = (df["Open"] / df["Open"].shift(-1)) -1
df["high_change"] = (df["High"] / df["High"].shift(-1)) -1
df["low_change"] = (df["Low"] / df["Low"].shift(-1)) -1
df["close_change"] = (df["Close"] / df["Close"].shift(-1)) - 1

df = df.drop("Unix Timestamp", axis=1)
df = df.drop("Date", axis=1)
df = df.drop("Volume BTC", axis=1)
df = df.drop("Volume ETH", axis=1)
df = df.drop("Volume LTC", axis=1)
df = df.drop("Volume USD", axis=1)
df.drop(df.tail(8).index,inplace=True)

df = df.dropna()

df

,Symbol,Open,High,Low,Close,open_change,high_change,low_change,close_change
0,BTCUSD,18168.59,18390.00,18100.00,18194.27,0.002070,0.011144,0.001045,0.001413
1,BTCUSD,18131.05,18187.32,18081.10,18168.59,-0.003849,-0.002197,0.001187,0.002070
2,BTCUSD,18201.11,18227.37,18059.67,18131.05,0.013638,-0.000418,0.006381,-0.003849
3,BTCUSD,17956.22,18234.99,17945.16,18201.11,-0.000905,0.011471,-0.000496,0.013638
4,BTCUSD,17972.49,18028.19,17954.06,17956.22,-0.000483,-0.001608,0.000862,-0.000905
...,...,...,...,...,...,...,...,...,...
88795,LTCUSD,40.05,40.05,40.05,40.05,-0.013547,-0.013547,0.011364,0.000000
88796,LTCUSD,40.60,40.60,39.60,40.05,0.038363,-0.013126,0.036649,-0.013547
88797,LTCUSD,39.10,41.14,38.20,40.60,-0.022744,0.013800,-0.023018,0.038363
88798,LTCUSD,40.01,40.58,39.10,39.10,-0.028412,-0.014570,-0.022500,-0.022744


SyntaxError: invalid syntax (<ipython-input-9-6f064e59bbb2>, line 1)

In [52]:
import requests
url = 'https://rest.coinapi.io/v1/exchangerate/BTC/EUR?invert=false'
headers = {'X-CoinAPI-Key' : 'E69DD5CD-B724-4683-A9F6-546DE5C3EB58'}
response = requests.get(url, headers=headers)

#print(response.text)


In [53]:
print(response.text)



#print(len(df.index))
#df.loc[[1]]


{
  "time": "2021-10-19T08:22:17.0000000Z",
  "asset_id_base": "BTC",
  "asset_id_quote": "EUR",
  "rate": 53270.928569553638954961350862,
  "src_side_base": [
    {
      "time": "2021-10-19T08:22:17.0000000Z",
      "asset": "XTZ",
      "rate": 62025.026342936219089854972116,
      "volume": 13196035.953724493443088481539
    },
    {
      "time": "2021-10-19T08:22:17.0000000Z",
      "asset": "LTC",
      "rate": 62018.643033587562180676692242,
      "volume": 42314609.284330085841166781101
    },
    {
      "time": "2021-10-19T08:22:17.0000000Z",
      "asset": "ATOM",
      "rate": 62060.612493287502840752488374,
      "volume": 13804396.107299994679488760745
    },
    {
      "time": "2021-10-19T08:22:17.0000000Z",
      "asset": "ETH",
      "rate": 62042.026977348305761768453052,
      "volume": 503354405.55753155344557438849
    },
    {
      "time": "2021-10-19T08:22:17.0000000Z",
      "asset": "ADA",
      "rate": 62033.944192209562584999592669,
      "volume": 9988341

In [22]:

X = []
Y = []

changes = df[["open_change", "high_change", "low_change", "close_change"]].to_numpy()


for i in range(0, len(changes) - 20):
    Y.append(changes[i])
    X.append(np.array(changes[i+1:i+21][::-1]))
    
X = np.array(X).reshape(-1, 20, 4)
Y = np.array(Y)
print(X.shape)
print(Y.shape)

(88780, 20, 4)
(88780, 4)


In [23]:
from keras.models import Sequential 
from keras.layers import LSTM

model = Sequential()
model.add(LSTM(4, input_shape=(20, 4)))

model.compile(optimizer="rmsprop", loss="mse")
model.fit(X, Y, batch_size=32, epochs=10)

Epoch 1/10
2775/2775 [==============================] - 26s 9ms/step - loss: 3.4190e-04
Epoch 2/10
2775/2775 [==============================] - 30s 11ms/step - loss: 3.1976e-04
Epoch 3/10
2775/2775 [==============================] - 18s 7ms/step - loss: 3.1585e-04
Epoch 4/10
2775/2775 [==============================] - 17s 6ms/step - loss: 3.1483e-04
Epoch 5/10
2775/2775 [==============================] - 17s 6ms/step - loss: 3.1439e-04
Epoch 6/10
2775/2775 [==============================] - 20s 7ms/step - loss: 3.1448e-04
Epoch 7/10
2775/2775 [==============================] - 22s 8ms/step - loss: 3.1412e-04
Epoch 8/10
2775/2775 [==============================] - 17s 6ms/step - loss: 3.1396e-04
Epoch 9/10
2775/2775 [==============================] - 17s 6ms/step - loss: 3.1354e-04
Epoch 10/10
2775/2775 [==============================] - 17s 6ms/step - loss: 3.1322e-04


In [24]:
predictions = model.predict(X)
print(predictions.shape)
df1 = pd.DataFrame(predictions, columns=['predictions_open', 'predictions_High', 'predictions_Low', 'predictions_Close'])
#df = pd.concat([df, df1])
df1

(88780, 4)


,predictions_open,predictions_High,predictions_Low,predictions_Close
0,0.001769,0.005282,0.000597,0.000155
1,-0.004143,0.001887,-0.000927,0.000200
2,0.013325,0.005667,0.009854,-0.000825
3,-0.001243,0.003323,-0.002434,0.000295
4,-0.000796,0.002716,-0.001947,0.000198
...,...,...,...,...
88775,0.005166,0.003727,0.008980,0.000096
88776,-0.008287,0.006850,-0.006566,0.001298
88777,-0.013242,-0.010095,-0.008291,0.003141
88778,-0.040757,-0.022588,-0.013922,0.002769


(443915, 13)